In [3]:
responses = []
import json
response_path = "/home/lym/GraphRAG4OralHealth/Benchmark/Knowledge Objectives/2d648ad1-917e-4be5-8fa2-befb0c3c1497_1744992291840_success.jsonl"
with open(response_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            print(line)
            data = json.loads(line)
            msg = data["response"]["body"]["choices"][0]["message"]
            responses.append({
                "id": data["custom_id"],
                "prediction": msg.get("content", "").strip(),
                "reason": msg.get("reasoning_content", "").strip()
            })
save_path="/home/lym/GraphRAG4OralHealth/Benchmark/Knowledge Objectives/success.jsonl"

#如果id不连续，补上空response
id = 1
for req in responses: 
    if req["id"] != "request-"+str(id):
        print(id)
        
        responses.insert(id-1, {
            "id": "request-"+str(id),
            "prediction": "",
            "reason": ""
        })
    id += 1
with open(save_path, "w", encoding="utf-8") as f:
    for response in responses:
        f.write(json.dumps(response, ensure_ascii=False) + "\n")

2159
2752
2785
2853
2861
2870
2881
2892
2897
2901
2903
2906
2910
2917
2925
2927
2931
2933
2936
2937
2946
2950
2952
2958
2960
2962
2964
2968
2974
2977
2983
2995
2998
